<a href="https://colab.research.google.com/github/rachotta3227/python/blob/main/Untitled23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter
import tkinter.messagebox
import random
FS = ("Times New Roman", 30)
FL = ("Times New Roman", 80)
BLACK = 1
WHITE = 2
mx = 0
my = 0
mc = 0
proc = 0
turn = 0
msg = ""
space = 0
color = [0]*2
who = ["あなた", "コンピュータ"]
board = []
for y in range(8):
board.append([0, 0, 0, 0, 0, 0, 0, 0])
def click(e):
global mx, my, mc
mc = 1
mx = int(e.x/80)
my = int(e.y/80)
if mx > 7:
mx = 7
if my > 7:
my = 7
def banmen():
cvs.delete("all")
cvs.create_text(320, 670, text=msg, fill="silver", font=FS)
for y in range(8):
for x in range(8):
X = x*80
Y = y*80
cvs.create_rectangle(X, Y, X+80, Y+80, outline="black")
if board[y][x] == BLACK:
cvs.create_oval(X+10, Y+10, X+70, Y+70, fill="black", width=0)
if board[y][x] == WHITE:
cvs.create_oval(X+10, Y+10, X+70, Y+70, fill="white", width=0)
cvs.update()
def ban_syokika():
global space
space = 60
for y in range(8):
for x in range(8):
board[y][x] = 0
board[3][4] = BLACK
board[4][3] = BLACK
board[3][3] = WHITE
board[4][4] = WHITE
# 石を打ち、相手の石をひっくり返す
def ishi_utsu(x, y, iro):
board[y][x] = iro
for dy in range(-1, 2):
for dx in range(-1, 2):
k = 0
sx = x
sy = y
while True:
sx += dx
sy += dy
if sx < 0 or sx > 7 or sy < 0 or sy > 7:
break
if board[sy][sx] == 0:
break
if board[sy][sx] == 3-iro:
k += 1
if board[sy][sx] == iro:
for i in range(k):
sx -= dx
sy -= dy
board[sy][sx] = iro
break
# そこに打つといくつ返せるか数える
def kaeseru(x, y, iro):
if board[y][x] > 0:
return -1 # 置けないマス
total = 0
for dy in range(-1, 2):
for dx in range(-1, 2):
k = 0
sx = x
sy = y
while True:
sx += dx
sy += dy
if sx < 0 or sx > 7 or sy < 0 or sy > 7:
break
if board[sy][sx] == 0:
break
if board[sy][sx] == 3-iro:
k += 1
if board[sy][sx] == iro:
total += k
break
return total
# 打てるマスがあるか調べる
def uteru_masu(iro):
for y in range(8):
for x in range(8):
if kaeseru(x, y, iro) > 0:
return True
return False
# 黒い石、白い石、いくつかあるか数える
def ishino_kazu():
b = 0
w = 0
for y in range(8):
for x in range(8):
if board[y][x] == BLACK:
b += 1
if board[y][x] == WHITE:
w += 1
return b, w
# 盤面の重み付け
banmen_weighting = [[30, -12, 0, -1, -1, 0, -12, 30], [-12, -15, -3, -3, -3, -3, -15, -12],
[0, -3, 0, -1, -1, 0, -3, 0],
[-1, -3, -1, -1, -1, -1, -3, -1],
[-1, -3, -1, -1, -1, -1, -3, -1],
[0, -3, 0, -1, -1, 0, -3, 0],
[-12, -15, -3, -3, -3, -3, -15, -12],
[30, -12, 0, -1, -1, 0, -12, 30]]
# 評価値を算出
def hyouka(ura_board, iro):
result=0
for i, y in enumerate(ura_board): # 行 for key, value in 元の配列 for n, x in enumerate(y): # 列
if x != 0:
if x == iro:
result += banmen_weighting[i][n]
else:
result -= banmen_weighting[i][n]
return result
# 裏盤面から次の裏盤面を返す
def ura_ishi_utsu(x, y, iro, ura_board):
ura_board[y][x] = iro
for dy in range(-1, 2):
for dx in range(-1, 2):
k = 0
sx = x
sy = y
while True:
sx += dx
sy += dy
if sx<0 or sx>7 or sy<0 or sy>7:
break
if ura_board[sy][sx]==0:
break
if ura_board[sy][sx]==3-iro:
k += 1
if ura_board[sy][sx]==iro:
for i in range(k):
sx -= dx
sy -= dy
ura_board[sy][sx] = iro
break
return ura_board
# 裏盤面からそこに打つといくつ返せるか数える
def ura_kaeseru(x, y, iro, ura_board):
if ura_board[y][x] > 0:
return False # 置けないマス
total = 0
for dy in range(-1, 2):
for dx in range(-1, 2):
k = 0
sx = x
sy = y
while True:
sx += dx
sy += dy
if sx < 0 or sx > 7 or sy < 0 or sy > 7:
break
if ura_board[sy][sx] == 0:
break
if ura_board[sy][sx] == 3-iro:
k += 1
if ura_board[sy][sx] == iro:
total += k
if total>0:
return True
break
return False
# 探索の深さ
SEARCH_LEVEL = 4
min = -999999
max = 999999
# コンピュータの思考ルーチン (import copyしてください)
def computer_0(u_turn):
alpha = min
beta = max
# AlpfaBeta法で探索
rx, ry = alphaBeta(u_turn, SEARCH_LEVEL, alpha, beta, copy.deepcopy(board)) return rx, ry
# alpha-beta
def alphaBeta(u_turn, level, alpha, beta, ura_board):
# ノードの評価値
value = 0
# 子ノードから伝搬してきた評価値
childValue = 0
# alphaBeta法で求めた最大の評価値を持つ場所
bestX = 0
bestY = 0
# 末端(葉, level == 0)だったら評価値を計算
if level == 0:
# 評価値を返す
return hyouka(ura_board, color[turn])
if u_turn == turn:
# 自分の手番では最大の評価値を見つけたいので最初に最小値をセットしておく value = min
else:
# 相手の手番では最小の評価値を見つけたいので最初に最大値をセットしておく value = max
# 裏盤面で打てるところはすべて試す
for i, y in enumerate(ura_board):
for n, x in enumerate(y):
if ura_kaeseru(n, i, color[u_turn], ura_board): # もし打てるますだったら # 試しに打ってみる
next_ura_board = ura_ishi_utsu(
n, i, color[u_turn], copy.deepcopy(ura_board))
# ターンを入れ替える 白→黒, 黒→白
next_turn = 1 - u_turn
# 子ノードの評価値を求める(再帰)
childValue = alphaBeta(
next_turn, level - 1, alpha, beta, next_ura_board)
# 子ノードとこのノードの評価値を比較する
if u_turn == turn: # 自分のノード
# 最大の評価値を選ぶ
if childValue > value:
value = childValue
alpha = value
bestX = n
bestY = i
# このノードの現在のvalueが
# 親から受け継いだβ値より大きかったら
# この枝が選ばれることはないのでこれ以上評価しない
# = forループをぬける
if value >= beta: # betaカット
return value
else: # 相手のノード
# 最小の評価値を選ぶ
if childValue < value:
value = childValue
beta = value
bestX = n
bestY = i
# このノードの現在のvalueが
# 親から受け継いだβ値より小さかったら
# この枝が選ばれることはないのでこれ以上評価しない
# = forループをぬける
if value <= alpha: # betaカット
return value
if value == max or value == min:
return hyouka(ura_board, color[turn])
if level == SEARCH_LEVEL:
print(f"bestX={bestX}, bestY={bestY}, value={value}")
return bestX, bestY
else:
return value
def main():
global mc, proc, turn, msg, space
banmen()
if proc == 0: # タイトル画面
msg = "先手、後手を選んでください"
cvs.create_text(320, 200, text="Reversi", fill="gold", font=FL) cvs.create_text(160, 440, text="先手(黒)", fill="lime", font=FS) cvs.create_text(480, 440, text="後手(白)", fill="lime", font=FS) if mc == 1: # ウィンドウをクリック
mc = 0
if (mx == 1 or mx == 2) and my == 5:
ban_syokika()
color[0] = BLACK
color[1] = WHITE
turn = 0
proc = 1
if (mx == 5 or mx == 6) and my == 5:
ban_syokika()
color[0] = WHITE
color[1] = BLACK
turn = 1
proc = 1
elif proc == 1: # どちらの番か表示
msg = "あなたの番です"
if turn == 1:
msg = "コンピュータ 考え中."
proc = 2
elif proc == 2: # 石を打つマスを決める
if turn == 0: # プレイヤー
if mc == 1:
mc = 0
if kaeseru(mx, my, color[turn]) > 0:
ishi_utsu(mx, my, color[turn])
space -= 1
proc = 3
else: # コンピュータ
cx, cy = computer_0(turn)
ishi_utsu(cx, cy, color[turn])
space -= 1
proc = 3
elif proc == 3: # 打つ番を交代
msg = ""
turn = 1-turn
proc = 4
elif proc == 4: # 打てるマスがあるか
if space == 0:
proc = 5
elif uteru_masu(BLACK) == False and uteru_masu(WHITE) == False: tkinter.messagebox.showinfo("", "どちらも打てないので終了です") proc = 5
elif uteru_masu(color[turn]) == False:
tkinter.messagebox.showinfo("", who[turn]+"は打てないのでパスです") proc = 3
else:
proc = 1
elif proc == 5: # 勝敗判定
b, w = ishino_kazu()
tkinter.messagebox.showinfo("終了", "黒={}、白={}".format(b, w)) if (color[0] == BLACK and b > w) or (color[0] == WHITE and w > b): tkinter.messagebox.showinfo("", "あなたの勝ち！")
elif (color[1] == BLACK and b > w) or (color[1] == WHITE and w > b): tkinter.messagebox.showinfo("", "コンピュータの勝ち！")
else:
tkinter.messagebox.showinfo("", "引き分け")
proc = 0
root.after(100, main)
root = tkinter.Tk()
root.title("リバーシ")
root.resizable(False, False)
root.bind("<Button>", click)
cvs = tkinter.Canvas(width=640, height=700, bg="green")


IndentationError: expected an indented block after 'for' statement on line 18 (<ipython-input-2-02e171f9fc3c>, line 19)